# This notebook is to generate summaries for the most common topics (extracted via the clustering script). These summaries can then be pre-loaded by the final model if needed, rather than having to generate them from scratch. Which ultimately saves time and efficiency.  

In [1]:
import os
import pickle
import json
import boto3
import logging
import yaml
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Any, Union
from tqdm import tqdm
from openai import OpenAI
from pinecone import Pinecone
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# ---------- Configure Logging ----------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('topic_summarization.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ---------- Config ----------
class Config:
    def __init__(self, config_file="config.yaml"):
        # Load configuration from YAML file
        self.config_data = self.load_config(config_file)
        
        # File paths
        self.CLUSTERED_VECTORS_PATH = "rizzbot_data/cleaned_clustered_vectors.pkl" # Path the the cleaned clustered vectors file
        self.TOPIC_MODEL_PATH = "rizzbot_data/bertopic_model"
        
        # S3 settings
        self.S3_BUCKET = "rizzbot-temp-storage"
        self.S3_PREFIX = "rizzbot/Summaries/"
        
        # Pinecone settings
        self.PINECONE_INDEX = "rizzbot-summaries"
        
        # Model settings
        self.EMBEDDING_MODEL = "all-MiniLM-L6-v2"
        self.SUMMARY_MODEL = "gpt-4o-mini"
        
        # Processing settings
        self.MAX_SUMMARY_WORDS = 500
        self.MAX_DOCS_PER_TOPIC = 50
        self.CHUNK_SIZE = 500
        
        # API Keys from config.yaml
        self.OPENAI_API_KEY = self.config_data.get('openai_api_key')
        self.PINECONE_API_KEY = self.config_data.get('pinecone_api_key')
        
        # Validate required keys
        self.validate_config()
    
    def load_config(self, config_file: str) -> dict:
        """Load configuration from YAML file"""
        try:
            with open(config_file, 'r') as f:
                config = yaml.safe_load(f)
            logger.info(f"Loaded configuration from {config_file}")
            return config
        except FileNotFoundError:
            logger.error(f"Configuration file {config_file} not found")
            raise
        except yaml.YAMLError as e:
            logger.error(f"Error parsing YAML file: {e}")
            raise
    
    def validate_config(self):
        """Validate that required API keys are present"""
        missing_keys = []
        
        if not self.OPENAI_API_KEY:
            missing_keys.append('openai_api_key')
        
        if not self.PINECONE_API_KEY:
            missing_keys.append('pinecone_api_key')
        
        if missing_keys:
            logger.error(f"Missing required API keys in config.yaml: {missing_keys}")
            raise ValueError(f"Missing required API keys: {', '.join(missing_keys)}")
        
        logger.info("All required API keys found in configuration")

class TopicSummarizer:
    def __init__(self, config_file="config.yaml"):
        self.config = Config(config_file)
        self.s3_client = None
        self.openai_client = None
        self.pinecone_client = None
        self.pinecone_index = None
        self.embedder = None
        self.topic_model = None
        self.clustered_vectors = None
        
    def initialize_clients(self):
        """Initialize all external service clients"""
        try:
            logger.info("Initializing clients...")
            
            # Initialize S3 client (uses AWS credentials from environment/AWS config)
            self.s3_client = boto3.client("s3")
            logger.info("S3 client initialized")
            
            # Initialize OpenAI client with API key from config
            self.openai_client = OpenAI(api_key=self.config.OPENAI_API_KEY)
            logger.info("OpenAI client initialized")
            
            # Test OpenAI connection
            try:
                test_response = self.openai_client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": "Hello"}],
                    max_tokens=5
                )
                logger.info("OpenAI API connection test successful")
            except Exception as e:
                logger.error(f"OpenAI API connection test failed: {e}")
                raise
            
            # Initialize Pinecone client with API key from config
            self.pinecone_client = Pinecone(api_key=self.config.PINECONE_API_KEY)
            self.pinecone_index = self.pinecone_client.Index(self.config.PINECONE_INDEX)
            logger.info("Pinecone client initialized")
            
            # Initialize sentence transformer (no API key needed)
            self.embedder = SentenceTransformer(self.config.EMBEDDING_MODEL)
            logger.info("Sentence transformer initialized")
            
            logger.info("All clients initialized successfully")
        except Exception as e:
            logger.error(f"Failed to initialize clients: {e}")
            raise
    
    def load_data(self):
    #Load clustered data and BERTopic model"""
        try:
            logger.info("Loading clustered vectors...")
            with open(self.config.CLUSTERED_VECTORS_PATH, "rb") as f:
                self.clustered_vectors = pickle.load(f)
            logger.info(f"Loaded clustered vectors: {len(self.clustered_vectors)} items")
            
            if isinstance(self.clustered_vectors, pd.DataFrame):
                logger.info(f"DataFrame shape: {self.clustered_vectors.shape}, columns: {list(self.clustered_vectors.columns)}")
            else:
                logger.info(f"Data type: {type(self.clustered_vectors)}")

            self.load_topic_model_first()
        except Exception as e:
            logger.error(f"Error loading data: {e}")
            raise

    
    def load_topic_model_first(self):
    #Load BERTopic model with re-attached embedding model
        try:
            logger.info(f"Loading BERTopic model from: {self.config.TOPIC_MODEL_PATH}")
            embedding_model = SentenceTransformer(self.config.EMBEDDING_MODEL)
            self.topic_model = BERTopic.load(self.config.TOPIC_MODEL_PATH, embedding_model=embedding_model)
            topics = self.topic_model.get_topics()
            logger.info(f"Loaded BERTopic model with {len(topics)} topics")
        except Exception as e:
            logger.error(f"Error loading BERTopic model: {e}")
            raise

    
    def group_documents_by_topic(self) -> Dict[int, List[str]]:
        logger.info("Grouping documents by topic...")
        topic_to_docs = {}

        if isinstance(self.clustered_vectors, pd.DataFrame):
            df = self.clustered_vectors
            if 'topic_id' not in df.columns or 'text' not in df.columns:
                raise ValueError("clustered_vectors must include 'topic_id' and 'text'")
            for topic_id, group in df.dropna(subset=['topic_id', 'text']).groupby('topic_id'):
                if topic_id == -1:
                    continue
                topic_to_docs[int(topic_id)] = group['text'].tolist()
        else:
            for item in self.clustered_vectors:
                if not isinstance(item, dict): continue
                topic_id = item.get("topic_id")
                text = item.get("text")
                if topic_id is not None and text and topic_id != -1:
                    topic_to_docs.setdefault(topic_id, []).append(text)

        logger.info(f"Grouped documents into {len(topic_to_docs)} topics")
        return topic_to_docs

    
    def generate_summary(self, topic_id: int, docs: List[str]) -> str:
        """Generate summary for a single topic with enhanced error handling"""
        try:
            combined_text = "\n\n".join(docs[:self.config.MAX_DOCS_PER_TOPIC])
            
            logger.info(f"Generating summary for topic {topic_id} with {len(docs)} documents")
            logger.info(f"Combined text length: {len(combined_text)} characters")
            
            prompt = (
                f"You are a helpful assistant. Write a concise and clear {self.config.MAX_SUMMARY_WORDS}-word summary "
                f"on the following topic using the provided documents:\n\n{combined_text}\n\n"
                f"Summary:"
            )
            
            response = self.openai_client.chat.completions.create(
                model=self.config.SUMMARY_MODEL,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
                max_tokens=1500
            )
            
            summary = response.choices[0].message.content.strip()
            logger.info(f"Successfully generated summary for topic {topic_id}: {len(summary)} characters")
            return summary
            
        except Exception as e:
            logger.error(f"Failed to generate summary for topic {topic_id}: {e}")
            raise
    
    def save_to_s3(self, topic_id: int, summary_text: str, run_number: int):
        """Save summary to S3 with run number"""
        try:
            s3_key = f"{self.config.S3_PREFIX}run_{run_number}/topic_{topic_id}.json"
            
            summary_data = {
                "topic_id": topic_id,
                "summary_text": summary_text,
                "source": "BERTopic",
                "run_number": run_number,
                "timestamp": datetime.now().isoformat(),
                "model_used": self.config.SUMMARY_MODEL
            }
            
            self.s3_client.put_object(
                Bucket=self.config.S3_BUCKET,
                Key=s3_key,
                Body=json.dumps(summary_data, indent=2),
                ContentType="application/json"
            )
            logger.info(f"Saved topic {topic_id} to S3: {s3_key}")
            
        except Exception as e:
            logger.error(f"Failed to save topic {topic_id} to S3: {e}")
            raise
    
    def save_to_pinecone(self, topic_id: int, summary_text: str, run_number: int):
        """Save summary chunks to Pinecone with run number"""
        try:
            chunks = [summary_text[i:i+self.config.CHUNK_SIZE] 
                     for i in range(0, len(summary_text), self.config.CHUNK_SIZE)]
            
            vectors_to_upsert = []
            for i, chunk in enumerate(chunks):
                embedding = self.embedder.encode(chunk).tolist()
                vector_id = f"summary-run{run_number}-{topic_id}-{i}"
                
                metadata = {
                    "type": "summary",
                    "topic_id": str(topic_id),
                    "chunk_id": i,
                    "source": "BERTopic",
                    "summary_quality": "v1.0",
                    "run_number": run_number,
                    "timestamp": datetime.now().isoformat(),
                    "model_used": self.config.SUMMARY_MODEL
                }
                
                vectors_to_upsert.append((vector_id, embedding, metadata))
            
            # Batch upsert for efficiency
            self.pinecone_index.upsert(vectors=vectors_to_upsert)
            logger.info(f"Saved {len(chunks)} chunks for topic {topic_id} to Pinecone")
            
        except Exception as e:
            logger.error(f"Failed to save topic {topic_id} to Pinecone: {e}")
            raise
    
    def save_summaries_locally(self, summaries: Dict[int, str], run_number: int):
        """Save summaries to local file for backup"""
        try:
            local_dir = f"rizzbot_data/summaries_run_{run_number}"
            os.makedirs(local_dir, exist_ok=True)
            
            # Save individual summaries
            for topic_id, summary_text in summaries.items():
                filename = os.path.join(local_dir, f"topic_{topic_id}.txt")
                with open(filename, 'w', encoding='utf-8') as f:
                    f.write(f"Topic ID: {topic_id}\n")
                    f.write(f"Run Number: {run_number}\n")
                    f.write(f"Timestamp: {datetime.now().isoformat()}\n")
                    f.write(f"Model: {self.config.SUMMARY_MODEL}\n")
                    f.write("-" * 50 + "\n")
                    f.write(summary_text)
            
            # Save consolidated file
            consolidated_file = os.path.join(local_dir, "all_summaries.json")
            with open(consolidated_file, 'w', encoding='utf-8') as f:
                summary_data = {
                    "run_number": run_number,
                    "timestamp": datetime.now().isoformat(),
                    "model_used": self.config.SUMMARY_MODEL,
                    "total_topics": len(summaries),
                    "summaries": summaries
                }
                json.dump(summary_data, f, indent=2, ensure_ascii=False)
            
            logger.info(f"Saved {len(summaries)} summaries locally in {local_dir}")
            
        except Exception as e:
            logger.error(f"Failed to save summaries locally: {e}")
            raise
    
    def run_full_summarization(self):
    #Run summarization for all topics once
        run_number = 1
        topic_to_docs = self.group_documents_by_topic()
        summaries = {}
        failed = []

        logger.info(f"Running summarization for {len(topic_to_docs)} topics...")

        for topic_id in tqdm(topic_to_docs, desc="Summarizing topics"):
            try:
                docs = topic_to_docs[topic_id]
                summary = self.generate_summary(topic_id, docs)
                summaries[topic_id] = summary
                self.save_to_s3(topic_id, summary, run_number)
                self.save_to_pinecone(topic_id, summary, run_number)
            except Exception as e:
                logger.error(f"Topic {topic_id} failed: {e}")
                failed.append(topic_id)

        self.save_summaries_locally(summaries, run_number)
        logger.info(f"Summary generation completed. Success: {len(summaries)} | Failed: {len(failed)}")

        return summaries

    
    def save_consolidated_results(self, all_results: Dict[int, Dict[int, str]]):
        """Save consolidated results from all runs"""
        try:
            consolidated_dir = "rizzbot_data/consolidated_results"
            os.makedirs(consolidated_dir, exist_ok=True)
            
            # Get total topics count properly
            topic_to_docs = self.group_documents_by_topic()
            total_topics = len(topic_to_docs)
            
            # Summary statistics
            stats = {
                "total_runs": self.config.NUM_TEST_RUNS,
                "timestamp": datetime.now().isoformat(),
                "model_used": self.config.SUMMARY_MODEL,
                "total_topics_available": total_topics,
                "run_statistics": {}
            }
            
            for run_number, summaries in all_results.items():
                stats["run_statistics"][run_number] = {
                    "topics_processed": len(summaries),
                    "success_rate": len(summaries) / total_topics if total_topics > 0 else 0
                }
            
            # Save statistics
            with open(os.path.join(consolidated_dir, "run_statistics.json"), 'w') as f:
                json.dump(stats, f, indent=2)
            
            # Save all results
            with open(os.path.join(consolidated_dir, "all_results.json"), 'w') as f:
                json.dump(all_results, f, indent=2, ensure_ascii=False)
            
            logger.info(f"Consolidated results saved in {consolidated_dir}")
            
        except Exception as e:
            logger.error(f"Failed to save consolidated results: {e}")
    
    def run(self):
        """Main execution method"""
        try:
            self.initialize_clients()
            self.load_data()
            return self.run_full_summarization()
        except Exception as e:
            logger.error(f"Fatal error during execution: {e}")
            raise


def main():
    """Main entry point"""
    try:
        # You can specify a different config file path if needed
        # summarizer = TopicSummarizer("path/to/your/config.yaml")
        summarizer = TopicSummarizer("config.yaml")  # Uses default "config.yaml" in current directory
        results = summarizer.run()
        
        # Print final summary
        total_successful = sum(len(summaries) for summaries in results.values())
        print(f"\n Final Results:")
        print(f"   Total summaries generated: {total_successful}")
        
    except Exception as e:
        logger.error(f"Script failed: {e}")
        raise

if __name__ == "__main__":
    main()

2025-06-30 18:16:09,513 - INFO - Loaded configuration from config.yaml
2025-06-30 18:16:09,515 - INFO - All required API keys found in configuration
2025-06-30 18:16:09,516 - INFO - Initializing clients...
2025-06-30 18:16:09,548 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-06-30 18:16:10,424 - INFO - S3 client initialized
2025-06-30 18:16:11,270 - INFO - OpenAI client initialized
2025-06-30 18:16:14,802 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-30 18:16:14,809 - INFO - OpenAI API connection test successful
2025-06-30 18:16:17,807 - INFO - Pinecone client initialized
2025-06-30 18:16:17,851 - INFO - Use pytorch device_name: cuda:0
2025-06-30 18:16:17,853 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-06-30 18:16:31,399 - INFO - Sentence transformer initialized
2025-06-30 18:16:31,400 - INFO - All clients initialized successfully
2025-06-30 18:16:31,401 - INFO - Loading clustere

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-06-30 18:24:32,912 - INFO - Saved 7 chunks for topic 53 to Pinecone
Summarizing topics: 100%|██████████| 45/45 [07:59<00:00, 10.67s/it]
2025-06-30 18:24:32,929 - INFO - Saved 45 summaries locally in rizzbot_data/summaries_run_1
2025-06-30 18:24:32,930 - INFO - Summary generation completed. Success: 45 | Failed: 0



 Final Results:
   Total summaries generated: 129067


In [ ]:
# Quick diagnostic script
import pickle

# Check your data structure
with open("rizzbot_data/full_vectors.pkl", "rb") as f:
    full_vectors = pickle.load(f)

print(f"Total items: {len(full_vectors)}")
print(f"full_vectors type: {type(full_vectors)}")
if isinstance(full_vectors, dict):
    first_item = next(iter(full_vectors.values()))
elif hasattr(full_vectors, "iloc"):  # DataFrame
    first_item = full_vectors.iloc[0]
else:
    first_item = full_vectors[0]
print(f"First item type: {type(first_item)}")
print(f"First item: {first_item}")

if hasattr(full_vectors, "columns"):
    print(f"Columns in DataFrame: {list(full_vectors.columns)}")
elif isinstance(full_vectors[0], dict):
    print(f"Keys in first item: {list(full_vectors[0].keys())}")

# Count items with topic_id
if hasattr(full_vectors, "topic_id"):
    count_with_topic_id = full_vectors["topic_id"].notnull().sum()
else:
    count_with_topic_id = sum(
        1 for item in full_vectors
        if isinstance(item, dict) and
        'topic_id' in item and
        item['topic_id'] is not None
    )
print(f"Items with valid topic_id: {count_with_topic_id}")
# ...existing code...

Total items: 741
full_vectors type: <class 'pandas.core.frame.DataFrame'>
First item type: <class 'pandas.core.series.Series'>
First item: id                                                    emb-0100
text          ,"Wait a second. You guys got stunt doubles? ...
embedding    [0.0493264534, 0.0287857763, -0.0159280337, 0....
Name: 0, dtype: object
Columns in DataFrame: ['id', 'text', 'embedding']
Items with valid topic_id: 0


In [ ]:
# Quick diagnostic script for clustered_vectors.pkl

import pickle

# Check your data structure
with open("rizzbot_data/clustered_vectors.pkl", "rb") as f:
    clustered_vectors = pickle.load(f)

print(f"Total items: {len(clustered_vectors)}")
print(f"clustered_vectors type: {type(clustered_vectors)}")
if isinstance(clustered_vectors, dict):
    first_item = next(iter(clustered_vectors.values()))
elif hasattr(clustered_vectors, "iloc"):  # DataFrame
    first_item = clustered_vectors.iloc[0]
else:
    first_item = clustered_vectors[0]
print(f"First item type: {type(first_item)}")
print(f"First item: {first_item}")

if hasattr(clustered_vectors, "columns"):
    print(f"Columns in DataFrame: {list(clustered_vectors.columns)}")
elif isinstance(clustered_vectors[0], dict):
    print(f"Keys in first item: {list(clustered_vectors[0].keys())}")

# Count items with topic_id
if hasattr(clustered_vectors, "topic_id"):
    count_with_topic_id = clustered_vectors["topic_id"].notnull().sum()
else:
    count_with_topic_id = sum(
        1 for item in clustered_vectors
        if isinstance(item, dict) and
        'topic_id' in item and
        item['topic_id'] is not None
    )
print(f"Items with valid topic_id: {count_with_topic_id}")

Total items: 741
clustered_vectors type: <class 'pandas.core.frame.DataFrame'>
First item type: <class 'pandas.core.series.Series'>
First item: id                                                      emb-0100
text            ,"Wait a second. You guys got stunt doubles? ...
embedding      [0.0493264534, 0.0287857763, -0.0159280337, 0....
cluster                                                        0
x                                                       7.796501
y                                                       14.66375
topic_num                                                     -1
topic_score                                             0.331495
Name: 0, dtype: object
Columns in DataFrame: ['id', 'text', 'embedding', 'cluster', 'x', 'y', 'topic_num', 'topic_score']
Items with valid topic_id: 0
